In [1]:
from h2_molecule import *
Precision.PRECISION = 5
Precision.update_threshold()

import pandas as pd
import seaborn as sns

In [2]:
# pauli matrices
I = np.array([[1, 0], [0, 1]])

X = np.array([[0, 1], [1, 0]])

Z = np.array([[1,0], [0,-1]])

Y = np.array([[0, complex(0, -1)], [complex(0, 1), 0]])

In [3]:
problem_instance = H2MoleculeInstance({0:0}, H2ExperimentID.P0_CliffordT)
initial_state = problem_instance.initial_state[0].to_np_array()

In [4]:
H = get_hamiltonian(H2ExperimentID.P0_CliffordT)

In [5]:
H.to_matrix()

array([[-1.82172106+0.j,  0.18177154+0.j],
       [ 0.18177154+0.j, -0.26673071+0.j]])

In [15]:
times = []
energies = []
for t in range(0, 14):
    times.append(t)
    state_t = schroedinger_equation(X, complex(0, -t), np.array([1/np.sqrt(2),1/np.sqrt(2)]))
    energies.append(get_energy(X, state_t))
    
state_t

array([0.5+0.5j, 0.5-0.5j])

In [3]:
eigenvalues, eigenvectors = np.linalg.eigh(X)

In [4]:
eigenvalues

array([-1.,  1.])

In [5]:
eigenvectors

array([[-0.70710678+0.j        , -0.70710678+0.j        ],
       [ 0.        +0.70710678j,  0.        -0.70710678j]])

In [6]:
get_energy(X, eigenvectors[0])

[-0.70710678+0.j -0.70710678+0.j]
after normalization: [-0.70710678+0.j -0.70710678+0.j]
q_dagger [-0.70710678-0.j -0.70710678-0.j]
H_q [0.+0.70710678j 0.-0.70710678j]
expectation value -4.266421588589642e-17j


0.0

In [13]:
get_fidelity(eigenvectors[0], state_t)

0.2500000000188757